In [18]:
#@title Input{display-mode: "form"}
Name = 'MAI Signature' #@param {type:"string"}
Moderne_abstrakte_Kunst = 5 #@param {type:"slider", min:0, max:20, step:1}
Zahl_der_Beispiele= 6 #@param {type:"slider", min:1, max:20, step:1}

In [ ]:
#@title Dateien vorbereiten {display-mode: "form"}
! git clone https://github.com/hansomatig/handwriting-synthesis.git
% cd handwriting-synthesis
! git checkout TFConV1toV2

In [ ]:
#@title Zusätzliche Module installieren{display-mode: "form"}
! pip install svgwrite
! pip install cairosvg

In [6]:
#@title Autoreload{display-mode: "form"}
%load_ext autoreload
%autoreload 2
from demo import Hand

In [7]:
#@title Tensorflow einrichten {display-mode: "form"}
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [ ]:
#@title Generator{display-mode: "form"}
from IPython.display import display, Image
import cairosvg
from keras.models import load_model
import PIL
import numpy as np

model = load_model('keras_model.h5')

lines = [" "*Moderne_abstrakte_Kunst + Name]
biases = [2.5 for i in lines]
styles = [4 for i in lines]
hand = Hand()
predictions = {}

for i in range(10):
    hand.write(
        filename=f'img/img{i}.svg',
        lines=lines,
        styles=styles,
        biases=biases,
    )
    cairosvg.svg2png(url=f'img/img{i}.svg', write_to=f'img/png{i}.png')

    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
    image = PIL.Image.open(f'img/png{i}.png')
    image = PIL.ImageOps.fit(image, (224, 224), PIL.Image.ANTIALIAS)
    image_array = np.asarray(image)
    normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
    data[0] = normalized_image_array

    prediction = model.predict(data)
    predictions[i] = prediction[0][0]

predictions = sorted(predictions.items(), key=lambda x: x[1], reverse=True)

In [ ]:
#@title Bestergebnisse anzeigen{display-mode: "form"}
import ipywidgets as widgets
from IPython.display import display
import functools

output = widgets.Output()
def on_button_clicked(b, path=''):
    with output:
        from google.colab import files
        files.download(filename=f'img/png{path}.png')

for i in range(Zahl_der_Beispiele):
    print(str(predictions[i][1]*100) + '%')
    display(Image(filename=f'img/png{predictions[i][0]}.png'))
    button = widgets.Button(description="Download", layout=widgets.Layout(width='50%', height='30px', display="inline-box", border="solid", margin='0 0 50px 0'))
    output = widgets.Output()
    button.on_click(functools.partial(on_button_clicked, path=predictions[i][0]))
    display(button, output)